# Análise dos logs HTTP do Servidor da Nasa

In [1]:
#inicialização 
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [2]:
#caminho do diretório do hdfs onde os arquivos de log estão armazenados
path = "/tmp/"

In [3]:
def parser(line):
    '''
    line: linha do arquivo de log
    retorna: retorna um spark Row contendo os campos host, date, url, code e bytes
    '''
    from pyspark.sql import Row
    import re
    from collections import defaultdict
    
    #cria um dicionario
    row = defaultdict() 
    
    #busca pelo padrão regex na linha
    host = re.search(r"(?P<host>\S+)\s+",line) 
    #Se o padrão for encontrado salva o resultado no dicionario, senão salva como Nulo 
    row['host'] = host.group('host') if host else None 
    
    #busca pelo padrão regex na linha
    date = re.search(r".+\[(?P<date>[\w/]+)[\w:]+\s[+\-]\d{4}\]+",line)
    #Se o padrão for encontrado salva o resultado no dicionario, senão salva como Nulo
    row['date'] = date.group('date') if date else None
    
    #busca pelo padrão regex na linha
    url = re.search(r".+\"\S+ (?P<url>\S+)\s+|\"",line)
    #Se o padrão for encontrado salva o resultado no dicionario, senão salva como Nulo
    row['url'] = url.group('url') if url else None
    
    #busca pelo padrão regex na linha
    code = re.search(r".+\" (?P<code>\d{3})+",line)
    #Se o padrão for encontrado salva o resultado no dicionario, senão salva como Nulo
    row['code'] =  code.group('code') if code else None
    
    #busca pelo padrão regex na linha
    byte = re.search(r".+\" \d{3} (?P<bytes>\d+)",line)
    #Se o padrão for encontrado salva o resultado como inteiro no dicionario, senão salva como 0
    row['byte'] = int(byte.group('bytes')) if byte else 0
    return Row(**row)

In [4]:
log_file1 = sc.textFile(path+"NASA_access_log_Jul95") # lê arquivo de log de Jul/95

In [5]:
df_log1= log_file1.map(parser).toDF() # aplica o paser em cada linha e cria um dataframe

In [6]:
df_log1.show(5,truncate=False)

+----+----+-----------+--------------------+-----------------------------------------------+
|byte|code|date       |host                |url                                            |
+----+----+-----------+--------------------+-----------------------------------------------+
|6245|200 |01/Jul/1995|199.72.81.55        |/history/apollo/                               |
|3985|200 |01/Jul/1995|unicomp6.unicomp.net|/shuttle/countdown/                            |
|4085|200 |01/Jul/1995|199.120.110.21      |/shuttle/missions/sts-73/mission-sts-73.html   |
|0   |304 |01/Jul/1995|burger.letters.com  |/shuttle/countdown/liftoff.html                |
|4179|200 |01/Jul/1995|199.120.110.21      |/shuttle/missions/sts-73/sts-73-patch-small.gif|
+----+----+-----------+--------------------+-----------------------------------------------+
only showing top 5 rows



In [7]:
#lê arquivo de log de Jul/95, aplica o paser em cada linha e cria um dataframe 
df_log2 = sc.textFile(path+"NASA_access_log_Aug95").map(parser).toDF()

In [8]:
df_log2.show(5,truncate=False)

+----+----+-----------+-----------------+-----------------------------------------------+
|byte|code|date       |host             |url                                            |
+----+----+-----------+-----------------+-----------------------------------------------+
|1839|200 |01/Aug/1995|in24.inetnebr.com|/shuttle/missions/sts-68/news/sts-68-mcc-05.txt|
|0   |304 |01/Aug/1995|uplherc.upl.com  |/                                              |
|0   |304 |01/Aug/1995|uplherc.upl.com  |/images/ksclogo-medium.gif                     |
|0   |304 |01/Aug/1995|uplherc.upl.com  |/images/MOSAIC-logosmall.gif                   |
|0   |304 |01/Aug/1995|uplherc.upl.com  |/images/USA-logosmall.gif                      |
+----+----+-----------+-----------------+-----------------------------------------------+
only showing top 5 rows



In [9]:
#Une os dois dataframes
df_log = df_log1.union(df_log2)

In [10]:
#matém o dataframe na memória
df_log.cache()

DataFrame[byte: bigint, code: string, date: string, host: string, url: string]

In [11]:
df_log.count()#total de linhas do dataframe

3461613

In [12]:
print("Total de valores nulos por coluna:")
for col in df_log.columns:
  print(" {}={}".format(col, df_log.filter(df_log[col].isNull() ).count()) )

Total de valores nulos por coluna:
 byte=0
 code=1
 date=1
 host=1
 url=60


In [13]:
#linhas em que a coluna host é nulo
df_log.filter(df_log["host"].isNull()).show()

+----+----+----+----+----+
|byte|code|date|host| url|
+----+----+----+----+----+
|   0|null|null|null|null|
+----+----+----+----+----+



O resultado acima representa a última linha do arquivo NASA_access_log_Jul95, que contém apenas a string "alyssa.p", logo não representa o log de uma requisição HTTP. Desse modo, essa linha será removida do dataframe.

In [14]:
df_log = df_log.filter(df_log["host"].isNotNull()) #remove linha inválida

In [15]:
df_log.filter(df_log["url"].isNull()).show(5,truncate=False)#linhas em que a coluna url é nulo

+----+----+-----------+---------------------------+----+
|byte|code|date       |host                       |url |
+----+----+-----------+---------------------------+----+
|0   |302 |11/Jul/1995|jumbo.jet.uk               |null|
|0   |302 |27/Jul/1995|drjo014a102.embratel.net.br|null|
|0   |302 |14/Aug/1995|203.16.174.5               |null|
|0   |302 |23/Aug/1995|mac391s.ksc.nasa.gov       |null|
|0   |302 |23/Aug/1995|mac391s.ksc.nasa.gov       |null|
+----+----+-----------+---------------------------+----+
only showing top 5 rows



Ao testar as regex a serem utilizadas na função parser identifiquei que os valores nulos da colunas url são decorrentes de requisições que não apresentam url de fato como mostrado no exemplo abaixo,ou url que apresentam caracteres com problemas de codificação([Mojibake]([https://pt.wikipedia.org/wiki/Mojibake)).
```
"GET  HTTP/1.0"
```
Nesses casos optei por deixar os valores como nulos.

## Número de hosts únicos

In [16]:
print("Número de hosts únicos = {}".format(df_log.select("host").distinct().count()))

Número de hosts únicos = 137978


## O total de erros 404

In [17]:
print("Total de erros 404 = {}".format(df_log.filter(df_log['code']=="404").count()))

Total de erros 404 = 20901


In [18]:
not_found = df_log.filter(df_log['code']=="404").cache()

## Os 5 URLs que mais causaram erro 404

In [19]:
not_found.groupBy("url").count().sort(desc("count")).show(5,truncate=False)

+--------------------------------------------+-----+
|url                                         |count|
+--------------------------------------------+-----+
|/pub/winvn/readme.txt                       |2004 |
|/pub/winvn/release.txt                      |1732 |
|/shuttle/missions/STS-69/mission-STS-69.html|682  |
|/shuttle/missions/sts-68/ksc-upclose.gif    |426  |
|/history/apollo/a-001/a-001-patch-small.gif |384  |
+--------------------------------------------+-----+
only showing top 5 rows



## Quantidade de erros 404 por dia

In [20]:
not_found.groupBy("date").count().show()

+-----------+-----+
|       date|count|
+-----------+-----+
|02/Jul/1995|  291|
|21/Aug/1995|  305|
|06/Aug/1995|  373|
|16/Jul/1995|  257|
|07/Aug/1995|  537|
|11/Aug/1995|  263|
|27/Jul/1995|  336|
|07/Jul/1995|  570|
|17/Jul/1995|  406|
|15/Jul/1995|  254|
|18/Jul/1995|  465|
|26/Jul/1995|  336|
|03/Aug/1995|  304|
|18/Aug/1995|  256|
|17/Aug/1995|  271|
|14/Aug/1995|  287|
|10/Jul/1995|  398|
|04/Jul/1995|  359|
|20/Aug/1995|  312|
|20/Jul/1995|  428|
+-----------+-----+
only showing top 20 rows



## Total de bytes retornados

In [21]:
print("Total de bytes retornados = {}".format(df_log.agg({"byte":"sum"}).collect()[0][0]) )

Total de bytes retornados = 65524314915
